In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w4p1/submit.csv
/kaggle/input/2022-ai-w4p1/train.csv
/kaggle/input/2022-ai-w4p1/test.csv


In [2]:
import random
import torch
import torch.optim as optim
import torch.nn as nn

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train = pd.read_csv('../input/2022-ai-w4p1/train.csv')
train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diabetes
0,1,95,60,18,58,23.9,0.260,22,0
1,5,105,72,29,325,36.9,0.159,28,0
2,0,135,68,42,250,42.3,0.365,24,1
3,4,131,68,21,166,33.1,0.160,28,0
4,1,103,30,38,83,43.3,0.183,33,0
...,...,...,...,...,...,...,...,...,...
532,5,139,64,35,140,28.6,0.411,26,0
533,1,96,122,0,0,22.4,0.207,27,0
534,10,101,86,37,0,45.6,1.136,38,1
535,0,141,0,0,0,42.4,0.205,29,1


# 데이터 정규화

# 데이터셋 가공하기

In [4]:
x_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]

In [5]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
x_train = scaler.fit_transform(x_train)

x_train = torch.FloatTensor(x_train)
x_train

tensor([[-0.4000, -0.5500, -0.7500,  ..., -0.8351, -0.3073, -0.4118],
        [ 0.4000, -0.3000,  0.0000,  ...,  0.5052, -0.5795, -0.0588],
        [-0.6000,  0.4500, -0.2500,  ...,  1.0619, -0.0243, -0.2941],
        ...,
        [ 1.4000, -0.4000,  0.8750,  ...,  1.4021,  2.0539,  0.5294],
        [-0.6000,  0.6000, -4.5000,  ...,  1.0722, -0.4555,  0.0000],
        [-0.6000,  0.2000,  1.5000,  ..., -0.9794, -0.3019, -0.4706]])

In [6]:
y_train = torch.FloatTensor(y_train.to_numpy())
y_train[:5]

tensor([0., 0., 1., 0., 0.])

In [7]:
print(x_train.shape)
print(y_train.shape)

torch.Size([537, 8])
torch.Size([537])


# 모델 학습하기

In [8]:
W = torch.zeros([8,1],requires_grad=True)
b = torch.zeros(1,requires_grad=True)

# optimzier 선택
optimizer = optim.Adam([W,b],lr=1e-1)

nb_epochs = 100000
for epoch in range(nb_epochs+1):

    
    #H(x)
    hypothesis = torch.sigmoid(x_train.matmul(W)+b)
    
    #cost
    loss = nn.BCELoss()
    cost = loss(hypothesis, y_train.unsqueeze(1))
    
    
    #
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%10000==0:
        print('Epoch: {:4d}/{}  Cost:{:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))


Epoch:    0/100000  Cost:0.693147
Epoch: 10000/100000  Cost:0.459388
Epoch: 20000/100000  Cost:0.459388
Epoch: 30000/100000  Cost:0.459388
Epoch: 40000/100000  Cost:0.459388
Epoch: 50000/100000  Cost:0.459388
Epoch: 60000/100000  Cost:0.459388
Epoch: 70000/100000  Cost:0.459411
Epoch: 80000/100000  Cost:0.459402
Epoch: 90000/100000  Cost:0.459507
Epoch: 100000/100000  Cost:0.459388


In [9]:
print(W)                                                                                                                                                                                      
print(b)

tensor([[ 0.2903],
        [ 1.4351],
        [-0.1734],
        [-0.0486],
        [-0.1276],
        [ 1.0577],
        [ 0.1566],
        [ 0.6113]], requires_grad=True)
tensor([-1.2367], requires_grad=True)


In [10]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
hypothesis[:5]

prediction = hypothesis >= torch.FloatTensor([0.5])

correct_prediction = prediction.float() == y_train.unsqueeze(1)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
accuracy

0.7821229050279329

# 모델 테스트하기

In [11]:
x_test = pd.read_csv('../input/2022-ai-w4p1/test.csv')
x_test

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,98,58,33,190,34.0,0.430,43
1,2,112,75,32,0,35.7,0.148,21
2,2,108,64,0,0,30.8,0.158,21
3,8,107,80,0,0,24.6,0.856,34
4,7,136,90,0,0,29.9,0.210,50
...,...,...,...,...,...,...,...,...
226,0,119,0,0,0,32.4,0.141,24
227,4,109,64,44,99,34.8,0.905,26
228,0,127,80,37,210,36.3,0.804,23
229,6,105,70,32,68,30.8,0.122,37


## 정규화

In [12]:
x_test = scaler.transform(x_test)
x_test

array([[ 0.6       , -0.475     , -0.875     , ...,  0.20618557,
         0.1509434 ,  0.82352941],
       [-0.2       , -0.125     ,  0.1875    , ...,  0.3814433 ,
        -0.60916442, -0.47058824],
       [-0.2       , -0.225     , -0.5       , ..., -0.12371134,
        -0.58221024, -0.47058824],
       ...,
       [-0.6       ,  0.25      ,  0.5       , ...,  0.44329897,
         1.15902965, -0.35294118],
       [ 0.6       , -0.3       , -0.125     , ..., -0.12371134,
        -0.67924528,  0.47058824],
       [ 0.4       , -1.        ,  0.625     , ...,  0.39175258,
        -0.58760108,  0.35294118]])

In [13]:
x_test = torch.FloatTensor(x_test)

In [14]:
hypothesis = torch.sigmoid(x_test.matmul(W)+b)
hypothesis[:5]

tensor([[0.2660],
        [0.1878],
        [0.1224],
        [0.1487],
        [0.5024]], grad_fn=<SliceBackward0>)

In [15]:
prediction = hypothesis >= torch.FloatTensor([0.5])
prediction = prediction.int()
prediction[:5]

tensor([[0],
        [0],
        [0],
        [0],
        [1]], dtype=torch.int32)

In [16]:
prediction = np.array(prediction)
prediction = pd.DataFrame(prediction,columns=['Diabetes'])
prediction

,Diabetes
0,0
1,0
2,0
3,0
4,1
...,...
226,0
227,0
228,0
229,0


# 제출용 만들기

In [17]:
submit = pd.read_csv('../input/2022-ai-w4p1/submit.csv')
submit

,ID,Diabetes
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
226,226,0
227,227,0
228,228,0
229,229,0


In [18]:
submit.update(prediction)
submit

,ID,Diabetes
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1
...,...,...
226,226,0
227,227,0
228,228,0
229,229,0


In [19]:
submit.to_csv('submit.csv',index=False)
submit

,ID,Diabetes
0,0,0
1,1,0
2,2,0
3,3,0
4,4,1
...,...,...
226,226,0
227,227,0
228,228,0
229,229,0
